#### UFLP Model

In [37]:
using DataFrames
using XLSX
import XLSX
xf = XLSX.readxlsx("10node.xlsx")
XLSX.sheetnames(xf)
d = xf["Sheet1"]
location = d["A2:E12"]
location = DataFrame(location[2:end,:],location[1,:])
distances = d["A15:K25"]
distances = DataFrame(distances[2:end,2:end], :auto)
location[1,"Fixed Cost"]


200

In [8]:

function calc_trans_cost(location,distances,status,assignment) #Given the opening of the inventory and the assignment, calculate the cost.



end

function UFLP_heuristic(location,distances,x,y)
    status = false(nrow(location)) #this is indicate whether a index is a inventory or not. If true it is an inventory, else it is a demand
    assignment = zeros(nrow(location)) #this is to indicate where are the demands assigned to.
    cost = typemax(Int64)#Initialize the cost to the largest possible number.
    temp_cost = typemax(Int64)#A temporary cost for each iteration, it will contain the minimum cost for each iteration.
    terminal = true #if all next possible opening will incur a larger cost we terminate.
    while terminal
        for i in 1 : length(status) 
            if !status[i] #try openning locations at the place where there is no inventory
                status[i] = true
                #Now we need to decide the assigment of the demands
                temp_assigment = zeros(nrow(location)) #we need a temp_assigment and only record it if it brings a lower cost.
                for j in length(status)
                    if !status[j]
                        assignment[j] = findfirst(x -> x==minimum(distances[j,status]),status) #assign the demand j to the inventory with the lowest distance
                    end
                end
                if temp_cost < sum(status.*location[1,"Fixed Cost"])+ #this is to calculate the best, need to add in transportation cost
                    #need to update tempcost and the assigment
                else
                    #we keep everything the same
                end
            end
        end
    end

end

UFLP_heuristic (generic function with 1 method)

In [33]:
a=trues(10)
a[1] = a[3] = a[7] = a[9] = false
b = zeros(5)
b .= typemax(Int64)
#findfirst(x -> x==minimum(distances[2,a]),a)
sum(a.*2)

12